In [1]:
from opensearchpy import OpenSearch
from opensearch_dsl import Search
import pandas as pd

In [2]:
from streamlit_chat import config
NCP_CONFIG = config.NCP_CONFIG
EMB_CONFIG = config.HCX_EMBEDDING
OSH_CONFIG = config.OPENSEARCH
EMB_MODEL_NAME = "clir-sts-dolphin"

In [3]:
host = OSH_CONFIG['HOST']
port = 9200
auth = (OSH_CONFIG['ID'], OSH_CONFIG['PASSWD'])


In [6]:
client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    timeout = 300
)

In [7]:
index_name = "ncp_edu_sc2-emb-test-index"
index_body = {
  "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 100
    }
  },
  "mappings": {
    "properties": {
        "origin-question":{
            "type": "text"
        },
        "origin-text":{
            "type": "text"
        },
        ("emb-model--" + EMB_MODEL_NAME): {
            "type": "text"
        },
        "vector": {
          "type": "knn_vector",
          "dimension": 1024,
          "method": {
            "name": "hnsw",
            "space_type": "cosinesimil",
            "engine": "nmslib",
            "parameters": {
              "ef_construction": 128,
              "m": 24
            }
          }
        }
    }
  }
}

In [8]:
response = client.indices.create(index=index_name, body=index_body)

In [9]:
from streamlit_chat.modules.HyperClovaX import HyperClovaX
hcx = HyperClovaX(clova_studio_api_key=EMB_CONFIG['X-NCP-CLOVASTUDIO-API-KEY'], apigw_api_key=EMB_CONFIG['X-NCP-APIGW-API-KEY'])
hcx.set_embedding(app_id=EMB_CONFIG['APP_ID'], model_name=EMB_MODEL_NAME)


In [11]:
df_csv = pd.read_csv(filepath_or_buffer='streamlit_chat/data/starcraft2.csv')

In [12]:
origin_question = []
origin_text = []
embedding_text = []
# print(len(df['Completion']))
for idx, line in df_csv.iterrows():
    print(f"{idx + 1} / {len(df_csv['Completion'])}")
    res = hcx.embedding(text = line['Completion'])
    if res.status_code == 200:
        embedding_result = res.json()['result']['embedding']
        origin_question.append(line['Text'])
        origin_text.append(line['Completion'])
        embedding_text.append(embedding_result)
    else:
        print(f'error : {line["Completion"]}')

1 / 246
2 / 246
3 / 246
4 / 246
5 / 246
6 / 246
7 / 246
8 / 246
9 / 246
10 / 246
11 / 246
12 / 246
13 / 246
14 / 246
15 / 246
16 / 246
17 / 246
18 / 246
19 / 246
20 / 246
21 / 246
22 / 246
23 / 246
24 / 246
25 / 246
26 / 246
27 / 246
28 / 246
29 / 246
30 / 246
31 / 246
32 / 246
33 / 246
34 / 246
35 / 246
36 / 246
37 / 246
38 / 246
39 / 246
40 / 246
41 / 246
42 / 246
43 / 246
44 / 246
45 / 246
46 / 246
47 / 246
48 / 246
49 / 246
50 / 246
51 / 246
52 / 246
53 / 246
54 / 246
55 / 246
56 / 246
57 / 246
58 / 246
59 / 246
60 / 246
61 / 246
62 / 246
63 / 246
64 / 246
65 / 246
66 / 246
67 / 246
68 / 246
69 / 246
70 / 246
71 / 246
72 / 246
73 / 246
74 / 246
75 / 246
76 / 246
77 / 246
78 / 246
79 / 246
80 / 246
81 / 246
82 / 246
83 / 246
84 / 246
85 / 246
86 / 246
87 / 246
88 / 246
89 / 246
90 / 246
91 / 246
92 / 246
93 / 246
94 / 246
95 / 246
96 / 246
97 / 246
98 / 246
99 / 246
100 / 246
101 / 246
102 / 246
103 / 246
104 / 246
105 / 246
106 / 246
107 / 246
108 / 246
109 / 246
110 / 246
111 / 24

In [13]:
if len(origin_text) == len(embedding_text):
    dt_dict = {'origin-question': origin_question, 'origin_text': origin_text, 'embedding_text': embedding_text}

    df_final = pd.DataFrame(dt_dict)


In [14]:
len(df_final)

246

In [15]:
## bulk insert
sc2_datas = []
for idx, line in df_final.iterrows():
    # print(line['origin_text'])
    # index_no = '{ "index" : { "_index" : "' + index_name + '", "_id" : "' + str(idx + 1) + '" } }'
    # data = '{ "origin-text": "' + line['origin_text'] + '", "sc2-vector": ' + str(line['embedding_text']) + ' }'
    
    index_no = { "index" : { "_index" : index_name, "_id" : (idx + 1)} }
    data =  {"origin-question": line['origin-question'], "origin-text": line['origin_text'], "vector": line['embedding_text']}
    sc2_datas.append(index_no)
    sc2_datas.append(data)

In [16]:
client.bulk(sc2_datas)

{'took': 354,
 'errors': False,
 'items': [{'index': {'_index': 'ncp_edu_sc2-emb-test-index',
    '_id': '1',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 2, 'failed': 0},
    '_seq_no': 0,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'ncp_edu_sc2-emb-test-index',
    '_id': '2',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 2, 'failed': 0},
    '_seq_no': 1,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'ncp_edu_sc2-emb-test-index',
    '_id': '3',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 2, 'failed': 0},
    '_seq_no': 2,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_index': 'ncp_edu_sc2-emb-test-index',
    '_id': '4',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 2, 'failed': 0},
    '_seq_no': 3,
    '_primary_term': 1,
    'status': 201}},
  {'index': {'_inde

In [21]:
query_text = "불곰의 공격력을 알려줘요"
query_text_emb = hcx.embedding(text=query_text)
query_text_emb = query_text_emb.json()['result']['embedding']

In [22]:
print(query_text_emb)

[0.93491656, 0.7083492, -0.82439053, 0.35874575, -0.85040796, 0.026163358, -0.043025196, 1.0553125, 1.0346802, 1.1619924, -0.650914, -0.29257426, -0.224392, -0.9783997, 0.09834356, 1.1005186, -0.33642775, -0.3015325, -0.37048274, -0.6367562, -1.0030706, -1.1851021, 0.38389277, -0.6254203, 1.4231606, 1.63995, -0.059149288, -1.2118444, -0.95250297, 1.7426734, -0.20686834, -0.70059466, -1.0677822, -0.22218376, -0.6744483, -1.1271979, 1.1884984, -0.28414056, -0.63359106, 0.12884559, -0.01201536, -0.62978536, 1.3499553, -1.6584936, 2.4498928, -0.09480341, -0.97144544, -1.1694154, -0.43689442, -0.96919453, -0.749929, -0.9520092, -0.68248034, 1.6066941, 1.0852158, -0.056008946, -1.2424107, -0.30174118, -1.0403626, -0.89467204, -0.90643805, 1.0945921, -0.14440347, 0.011660705, 0.67052203, -0.8028651, 0.10702482, -0.4175854, 1.2125826, 1.2770517, -1.1894403, -0.0017690156, -0.86484534, -0.73776513, -2.083998, 0.22875322, 0.30575988, -0.31935877, -0.23380452, 0.014689615, -0.33703968, -0.2865790

In [23]:
## Text Search
search_body_text_t = {
  "size": 10,
  "query": {
    "match": {
      "origin_text": query_text
    }
  }
}
search_result_text_t = client.search(index=index_name, body=search_body_text_t)
print(len(search_result_text_t["hits"]["hits"]))
for s_text_t in search_result_text_t["hits"]["hits"]:
    print(s_text_t)
    print(f"{s_text_t['_score']} | {s_text_t['_source']['origin-text']}")

0


In [24]:
## Text Search
search_body_text_q = {
  "size": 10,
  "query": {
    "match": {
      "origin-question": query_text
    }
  }
}
search_result_text_q = client.search(index=index_name, body=search_body_text_q)
print(len(search_result_text_q["hits"]["hits"]))
for s_text_q in search_result_text_q["hits"]["hits"]:
    print(s_text_q)
    print(f"{s_text_q['_score']} | {s_text_q['_source']['origin-text']}")

5
{'_index': 'ncp_edu_sc2-emb-test-index', '_id': '25', '_score': 4.6622844, '_source': {'origin-question': '불곰의 정보', 'origin-text': '불곰(Marauder)는 테란의 중돌격 보병 유닛 입니다. 응징자 유탄(Punisher Grenades)로 적을 공격하며, 중장갑 상대의 적에게 보다 큰 피해를 입힙니다.\n병영에 부착된 기술실에서 충격탄(Concussive Shells) 연구가 완료되면, 불곰의 응징자 유탄이 적을 일시적으로 느리게 만듭니다. (거대 유닛은 면역)', 'vector': [0.033098537, -0.08931482, -1.4836656, 0.010223052, -0.98053813, 0.17101067, 0.11375199, 1.5409106, 1.6841818, 1.1990383, 0.09654522, 0.5535445, -0.40943304, -1.6992675, 0.8703241, 1.1222498, -0.29309046, -0.2309394, -0.9467727, 0.48505932, 0.20260587, -0.48926398, -0.09958619, -1.1269984, 0.8225925, 0.8436089, -0.06530215, -0.6061037, -0.7801258, 2.1128008, -0.19166678, -0.21952447, -1.3833954, 0.4410267, -0.7162656, -1.3868091, 1.127666, -0.77503186, -1.0052383, 0.1423569, -0.017801268, -0.7798249, 0.6277135, -1.0047272, 2.545027, 0.27202564, -0.7667862, -1.7197127, -0.16584398, 0.23476714, -1.6042755, -0.46989933, -1.9913551, 1.4773569, 1.5619156, 0.375015

In [26]:

## Normal KNN
search_body = {
  "size": 15,
  "query": {
    "knn": {
      "vector": {
        "vector": query_text_emb,
        "k": 15
      }
    }
  }
}
search_result = client.search(index=index_name, body=search_body)
print(len(search_result["hits"]["hits"]))
for s in search_result["hits"]["hits"]:
    print(f"{s['_score']} | {s['_source']['origin-text']}")

15
0.7754099 | 스타크래프트2에 등장하는 불곰(Marauder)은 테란의 중돌격 보병으로 응진자 유탄(Punisher Grenades)을 사용하며, 중장갑 상대의 적에게 큰 피해를 입히는 보병 유닛 입니다.
높은 체력을 보유하고 있어 체력이 낮은 해병을 보호하며 전투가 가능합니다.
0.76708484 | 불곰(Marauder)는 테란의 중돌격 보병 유닛 입니다. 응징자 유탄(Punisher Grenades)로 적을 공격하며, 중장갑 상대의 적에게 보다 큰 피해를 입힙니다.
병영에 부착된 기술실에서 충격탄(Concussive Shells) 연구가 완료되면, 불곰의 응징자 유탄이 적을 일시적으로 느리게 만듭니다. (거대 유닛은 면역)
0.76502085 | 화염 기갑병은 하나의 기본 능력이 존재합니다.
- 화염차 모드 : 화염 기갑병을 기동력이 높은 화염차로 변신시킵니다. 화염차는 일직선의 화염 공격을 합니다.
0.75396264 | 불곰은 높은 체력을 보유하고 있지만 지상 공격만 가능하여, 해병 부대와 조합하여 사용하는 것이 좋습니다.
- 생명력 : 125
- 방어력 : 1(+1)
- 특성 : 중장갑-생체
0.7460867 | 화염차는 낮은 체력을 보유했지만 빠른 기동성으로 적들에게 혼란을 주기 좋고, 정찰 및 게릴라에 특화되어 있습니다. 또한 화염기갑병(Hellbat)으로 변신하여 생명력을 약간 높이고 공격력을 크게 증가시킬 수 있습니다.
0.7310183 | 화염차는 한가지 기본 능력이 존재합니다.
- 화염 기갑병 모드 : 화염차를 근거리 전투 유닛인 화염 기갑병으로 변신시킵니다. 화염 기갑병은 전방에 작은 부채꼴 형태의 화염 공격을 합니다.
0.72997224 | 불곰에 적용 가능한 업그레이드는 4가지가 있습니다.
- 전투 자극제 사용(Use Stimpack) : 몇 초 동안 이동 및 공격속도를 크게 향상스키지만, 사용 즉시 생명력 20을 잃습니다.
- 충격탄(Concussive Shells) : 불곰이 명중시킨 대상의 움직임이 일시적으로 느려집

In [27]:
query_text = "바이킹의 공격력을 알려주세요"
query_text_emb = hcx.embedding(text=query_text)
query_text_emb = query_text_emb.json()['result']['embedding']

## Hybrid Search
search_body = {
  "_source": {
    "exclude": [
      "vector"
    ]
  },
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "origin-question": {
              "query": query_text
            }
          }
        },
        {
          "match": {
            "origin_text": {
              "query": query_text
            }
          }
        },
        {
          "knn": {
            "vector": {
              "vector": query_text_emb,
              "k": 10
            }
          }
        }
      ]
    }
  },"size":10,
  "search_pipeline" : {
      "phase_results_processors": [
      {
        "normalization-processor": {
          "normalization": {
            "technique": "min_max"
          },
          "combination": {
            "technique": "arithmetic_mean",
            "parameters": {
              "weights": [
                0.2,
                0.1,
                0.7
              ]
            }
          }
        }
      }
    ]
  }
}
print(index_name)
# search_params = {
#     "search_pipeline" : "cw-nlp-search-pipeline"
# }
#search_result = client.search(index=index_name, body=search_body, params=search_params)

search_result = client.search(index=index_name, body=search_body)
print(len(search_result["hits"]["hits"]))
for s in search_result["hits"]["hits"]:
    print(f"{s['_score']} | {s['_source']['origin-text']}")

ncp_edu_sc2-emb-test-index
10
0.7359637 | 바이킹은 높은 체력을 가진 기계 유닛으로 다양한 목적으로 사용 가능합니다.
- 생명력 : 135
- 방어력 : 0(+1)
- 특성 : 중장갑-기계
0.7 | 바이킹은 3가지 적용 가능한 업그레이드가 있습니다.
- 지능형 제어 장치 : 바이킹의 전투 모드를 빠르게 전환합니다.
- 우주선 무기 업그레이드 1, 2, 3 단계 : 바이킹의 공격력을 증가시킵니다.
- 차량 및 우주선 장갑 업그레이드 1, 2, 3 단계 : 바이킹의 방어력을 증가시킵니다.
0.67833734 | 바이킹은 두가지 기본 능력이 존재합니다.
- 돌격 모드 : 바이킹을 돌격 모드로 전환합니다. 지상으로 이동하며 지상 목표물만 공격합니다.
- 전투기 모드 : 바이킹을 전투기 모드로 전환합니다. 공중으로 이동하며 공중 목표물만 공격합니다.
0.62398934 | 바이킹의 공격 능력은 아래와 같습니다.

랜저 유도탄(Lanzer Torpedoes)을 사용하는 공중 공격 모드(전투기 모드)
- 공격력 : 10(+1) x 2 / 중장갑 상대 : 14(+1) x 2
- 사거리 : 9
- 공격 속도 : 2(아주빠름 1.43)
- 대상 : 공중 목표물 

개틀링 포(Gatling Cannon)을 사용하는 지상 공격 모드(돌격 모드)
- 공격력 : 12(+1) / 기계 상대 : 20(+2)
- 사거리 : 6
- 공격 속도 : 1(아주 빠름 기준 0.71)
- 대상 : 지상 목표물
0.54111195 | 바이킹(Viking)은 스타크래프트2에 등장하는 테란 공중 유닛 입니다. 공중을 공격할 수 있는 전투기 모드와, 지상에 내려와 지상 유닛을 공격할 수 있는 돌격 모드 전환이 가능합니다.
0.28512767 | 스타크래프트2에 등장하는 테란 공중유닛 정보를 알려드리겠습니다.
- 바이킹(Viking) : 튼튼한 전투 지원기 입니다. 강력한 대주력함 공중 미사일을 탑재하고 있습니다. 돌격 모드로 변신하여 지상에 내려와 지상 유닛을 공격할 수 있